# NLP PROJECT FOR UE16CS333 BY TEAM 13
## Team Members :
## Abhishek Narayanan (01FB16ECS016)
## Abhishek Prasad (01FB16ECS017)
## Abijna Rao (01FB16ECS019)

# Ensemble Model Using Decision Tree Classifier, Naive Bayes Classifier And Random Forest Classifier

### This notebok compares our approach with the baseline models of the aforementioned reserach paper cited
### The ensemble learner is our added novelty

In [39]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

## Loading feature vectors along with tags

In [63]:
X = pd.read_csv('Twitterdata/featureVectors.csv')
print(list(X))
y = X['word.Tag']

# removing the Tag column from X to keep it as feature only.
X.drop('word.Tag', axis=1, inplace=True)

# handelling the NaN and inf values in the dataset
X=X.astype('float32')
y=y.astype('float32')
X = np.nan_to_num(X)

#split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

['+1:word', '+1:word.1stUpper()', '+1:word.isAlpha()', '+1:word.isdigit()', '+1:word.istitle()', '+1:word.isupper()', '+1:word.lower()', '+1:word.startsWith#()', '+1:word.startsWith@()', 'BOS', '-1:word', '-1:word.1stUpper()', '-1:word.isAlpha()', '-1:word.isdigit()', '-1:word.istitle()', '-1:word.isupper()', '-1:word.lower()', '-1:word.startsWith#()', '-1:word.startsWith@()', 'EOS', 'bias', 'word', 'word.1stUpper()', 'word.isAlpha()', 'word.isdigit()', 'word.istitle()', 'word.isupper()', 'word.lower()', 'word.startsWith#()', 'word.startsWith@()', 'word[-2:]', 'word[-3:]', 'word.Tag']


## Decision Tree Classifier

In [47]:
dtc = DecisionTreeClassifier(max_depth=32, class_weight="balanced")#[{0:1,1:1}, {0:1,1:50}, {0:1,1:18},{0:1,1:1940}, {0:1,1:70},{0:1,1:3},{0:1,1:25}])

# fit
dtc.fit(X_train, y_train)

# predict
y_pred = dtc.predict(X_test)
target_names = ['I-Loc', 'B-Org', 'I-Per', 'Other', 'B-Per', 'I-Org', 'B-Loc']

# print
print ("Results for Decision tree..")

print(classification_report(y_test, y_pred, target_names=target_names))


# f1 score
score = f1_score(y_pred, y_test, average='weighted')
print ("Decision Tree F1 score: {:.4f}".format(score))

Results for Decision tree..
              precision    recall  f1-score   support

       I-Loc       0.17      0.20      0.18        10
       B-Org       0.50      0.57      0.53       350
       I-Per       0.32      0.35      0.33       153
       Other       0.97      0.96      0.97     16653
       B-Per       0.61      0.60      0.60       645
       I-Org       0.33      0.26      0.29        23
       B-Loc       0.42      0.47      0.45       202

   micro avg       0.93      0.93      0.93     18036
   macro avg       0.47      0.49      0.48     18036
weighted avg       0.93      0.93      0.93     18036

Decision Tree F1 score: 0.9301


## Naive Bayes Model

In [48]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print ("Results for Naive Bayes...")
y_pred = gnb.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

# f1 score
score = f1_score(y_pred, y_test, average='weighted')
print ("Naive Bayes F1 score: {:.4f}".format(score))

Results for Naive Bayes...
              precision    recall  f1-score   support

       I-Loc       0.01      0.20      0.03        10
       B-Org       0.16      0.20      0.18       350
       I-Per       0.05      0.15      0.08       153
       Other       0.97      0.78      0.86     16653
       B-Per       0.27      0.45      0.34       645
       I-Org       0.00      0.35      0.01        23
       B-Loc       0.06      0.26      0.09       202

   micro avg       0.74      0.74      0.74     18036
   macro avg       0.22      0.34      0.23     18036
weighted avg       0.91      0.74      0.81     18036

Naive Bayes F1 score: 0.6685


In [58]:
clf = RandomForestClassifier(max_depth=10)
clf.fit(X_train, y_train)
print ("Results for Random Forest...")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

# f1 score
score = f1_score(y_pred, y_test, average='weighted')
print ("random Forest F1 score: {:.4f}".format(score))

Results for Random Forest...
              precision    recall  f1-score   support

       I-Loc       0.00      0.00      0.00        10
       B-Org       0.85      0.13      0.23       350
       I-Per       0.67      0.03      0.05       153
       Other       0.94      1.00      0.97     16653
       B-Per       0.79      0.36      0.50       645
       I-Org       0.00      0.00      0.00        23
       B-Loc       0.00      0.00      0.00       202

   micro avg       0.94      0.94      0.94     18036
   macro avg       0.46      0.22      0.25     18036
weighted avg       0.92      0.94      0.92     18036

random Forest F1 score: 0.9569


In [57]:
from sklearn.ensemble import VotingClassifier

#create a dictionary of our models
estimators=[('decision_tree', dtc), ('naive_bayes', gnb), ('random_forest', clf)]

#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

#fit model to training data
ensemble.fit(X_train, y_train)

print ("Results for Random Forest...")
#test our model on the test data
y_predict = ensemble.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))

# f1 score
score = f1_score(y_pred, y_test, average='weighted')
print ("Ensemble Model F1 score: {:.4f}".format(score))

Results for Random Forest...
              precision    recall  f1-score   support

       I-Loc       0.00      0.00      0.00        10
       B-Org       0.96      0.26      0.41       350
       I-Per       0.80      0.03      0.05       153
       Other       0.94      1.00      0.97     16653
       B-Per       0.82      0.38      0.52       645
       I-Org       0.00      0.00      0.00        23
       B-Loc       1.00      0.02      0.04       202

   micro avg       0.94      0.94      0.94     18036
   macro avg       0.65      0.24      0.28     18036
weighted avg       0.94      0.94      0.92     18036

Ensemble Model F1 score: 0.9583
